# Prediction of Early-Stage Melanoma Recurrence Using Clinical and Histopathologic Features
## Time to event prediction

Guihong Wan, July/27/2022     
Massachusetts General Hospital, Harvard Medical School

In [ ]:
run 'header.py'

In [ ]:
run 'Utils.py'

In [ ]:
def score_survival_model(model, X, y):
    '''
    Use concordance index to tune the parameters
    '''
    prediction = model.predict(X)
    result = concordance_index_censored(y['Recurrence'], y['Dia2Recur'], prediction)
    return result[0]

## 1. Data preprocessing

In [ ]:
# Use the sample data. Some code is not runnable due to small sample size.
melanoma_data = pd.read_csv("data/melanoma_example_v1.0.csv") 

In [ ]:
melanoma_data = melanoma_data.drop(columns = ['MelanomaID']+ nomi_features)
melanoma_data.tail()

In [ ]:
all_features = melanoma_data.columns.values.tolist()
print(all_features, len(all_features))

### MGH:DFCI split

In [ ]:
DEMOGRAPHCIS = False # DEMOGRAPHCIS only
MEDICAL = False # DEMOGRAPHCIS + meidcal history only
ALL = True # All features

In [ ]:
assert sum([DEMOGRAPHCIS, MEDICAL, ALL]) == 1

In [ ]:
if DEMOGRAPHCIS:
    combo = []
    for col in list(melanoma_data.columns):
        for target in demographic_features:
            if target in col:
                combo.append(col)

if MEDICAL:
    combo = []
    for col in list(melanoma_data.columns):
        for target in demographic_features + medical_features:
            if target in col:
                combo.append(col)

In [ ]:
if ALL:
    melanoma_data_cut = melanoma_data
else:
    melanoma_data_cut = melanoma_data[combo+['Site', 'Recurrence', 'Dia2Recur']]

data_MGH = melanoma_data_cut[melanoma_data_cut['Site'] == "MGB"]
data_DFCI = melanoma_data_cut[melanoma_data_cut['Site'] == "DFCI"]

data_MGH = data_MGH.drop(columns = ['Site'])
data_DFCI = data_DFCI.drop(columns = ['Site'])

print(data_MGH['Recurrence'].value_counts(), "\n")
print(data_DFCI['Recurrence'].value_counts(), "\n")

In [ ]:
X_mgh, y_mgh = get_x_y(data_MGH, attr_labels=['Recurrence','Dia2Recur'], pos_label=True, survival=True)

X_dfci, y_dfci = get_x_y(data_DFCI, attr_labels=['Recurrence','Dia2Recur'], pos_label=True, survival=True)

scaler = StandardScaler()
X_mgh = scaler.fit_transform(X_mgh)
X_dfci = scaler.fit_transform(X_dfci)

In [ ]:
nTrials = 50
nFold = 5

## 2. Internal validation
### RandomSurvivalForest

In [ ]:
if True:
    model = RandomSurvivalForest(bootstrap=False)

    param_grid = {'n_estimators': [100],
                  'min_samples_split': [4, 5, 8, 10, 12],
                  'min_samples_leaf': [4, 5,  8, 10, 12]
                 }
    cv = RepeatedStratifiedKFold(n_splits=nFold, n_repeats=1, random_state=42)
    gsearch = GridSearchCV(model, param_grid, scoring=score_survival_model, cv=cv)
    gsearch = gsearch.fit(X_mgh, y_mgh)

    print(gsearch.best_params_, gsearch.best_score_)
gsearch_RF = gsearch

In [ ]:
cindex_list = []
mAUC_list = []
for trial in range(nTrials):
    # 5-cross validation
    cv = RepeatedStratifiedKFold(n_splits=nFold, n_repeats=1)
    for train_index, test_index in cv.split(X_mgh, y_mgh):
        X_train, X_val = X_mgh[train_index], X_mgh[test_index]
        y_train, y_val = y_mgh[train_index], y_mgh[test_index]
        
        model = RandomSurvivalForest(n_estimators=100,
                              min_samples_split=gsearch_RF.best_params_['min_samples_split'],
                              min_samples_leaf=gsearch_RF.best_params_['min_samples_leaf'])
        
        model.fit(X_train, y_train)
        
        cindex = model.score(X_val, y_val)
        cindex_list.append(cindex)
        
        chf_funcs = model.predict_cumulative_hazard_function(X_val, return_array=False)
        
        times_val = list(chf_funcs[0].x)
        times_val.sort()
        times_test = [t for e,t in y_val]
        times_test.sort()
        times =[]
        for t in times_val:
            if t >= times_test[0] and t < times_test[-1]:
                times.append(t)
                
        risk_scores = np.row_stack([chf(times) for chf in chf_funcs])
        aucs, mean_auc = cumulative_dynamic_auc(y_train, y_val, risk_scores, times)
        mAUC_list.append(mean_auc)
        
        if trial%10 == 0: print("---" + str(trial) + ": cindex", cindex, " mean_auc:", mean_auc)

In [ ]:
getmeanAndCI(cindex_list, name="C-Index")
getmeanAndCI(cindex_list_RF, name="C-Index")
_,p = st.ttest_ind(cindex_list, cindex_list_RF)
print("acc %.3f" % p)
print()

getmeanAndCI(mAUC_list, name="Mean-AUC")
getmeanAndCI(mAUC_list_RF, name="Mean-AUC")
_,p = st.ttest_ind(mAUC_list, mAUC_list_RF)
print("acc %.3f" % p)
print()

In [ ]:
# when only negative/positive in the split, AUC is not avaialble.
# You may consider a smaller number of folds.
mAUC_list = [x for x in mAUC_list if math.isnan(x) == False]
if DEMOGRAPHCIS:
    cindex_mu_RF_demo, cindex_ci_RF_demo = getmeanAndCI(cindex_list, name="C-Index")
    mAUC_mu_RF_demo, mAUC_ci_RF_demo = getmeanAndCI(mAUC_list, name="Mean-AUC")
    cindex_list_RF_demo = cindex_list
    mAUC_list_RF_demo = mAUC_list
elif MEDICAL:
    cindex_mu_RF_med, cindex_ci_RF_med = getmeanAndCI(cindex_list, name="C-Index")
    mAUC_mu_RF_med, mAUC_ci_RF_med = getmeanAndCI(mAUC_list, name="Mean-AUC")
    cindex_list_RF_med = cindex_list
    mAUC_list_RF_med = mAUC_list
else:
    cindex_mu_RF, cindex_ci_RF = getmeanAndCI(cindex_list, name="C-Index")
    mAUC_mu_RF, mAUC_ci_RF = getmeanAndCI(mAUC_list, name="Mean-AUC")
    cindex_list_RF = cindex_list
    mAUC_list_RF = mAUC_list

### GradientBoostingSurvivalAnalysis

In [ ]:
if True:
    model = GradientBoostingSurvivalAnalysis(loss = 'coxph')

    param_grid = {'n_estimators': [100, 150],
                  'learning_rate': [0.1, 0.2, 0.3, 0.4 0.7, 0.9, 1.0],
                  'max_depth': [1,2]
                 }
    cv = RepeatedStratifiedKFold(n_splits=nFold, n_repeats=1, random_state=42)
    gsearch = GridSearchCV(model, param_grid, scoring=score_survival_model,cv=cv)
    gsearch = gsearch.fit(X_mgh, y_mgh)

    print(gsearch.best_params_, gsearch.best_score_)
    gsearch_GB = gsearch

In [ ]:
cindex_list = []
mAUC_list = []
for trial in range(nTrials):
    # 5-cross validation
    cv = RepeatedStratifiedKFold(n_splits=nFold, n_repeats=1)
    for train_index, test_index in cv.split(X_mgh, y_mgh):
        X_train, X_val = X_mgh[train_index], X_mgh[test_index]
        y_train, y_val = y_mgh[train_index], y_mgh[test_index]
        
        model = GradientBoostingSurvivalAnalysis(
            loss = 'coxph', 
            n_estimators=gsearch_GB.best_params_['n_estimators'], 
            learning_rate=gsearch_GB.best_params_['learning_rate'], 
            max_depth=gsearch_GB.best_params_['max_depth']
        )
        
        model.fit(X_train, y_train)
        
        cindex = model.score(X_val, y_val)
        cindex_list.append(cindex)
        
        chf_funcs = model.predict_cumulative_hazard_function(X_val)
        
        times_val = list(chf_funcs[0].x)
        times_val.sort()
        times_test = [t for e,t in y_val]
        times_test.sort()
        times =[]
        for t in times_val:
            if t >= times_test[0] and t < times_test[-1]:
                times.append(t)

        risk_scores = np.row_stack([chf(times) for chf in chf_funcs])
        aucs, mean_auc = cumulative_dynamic_auc(y_train, y_val, risk_scores, times)
        mAUC_list.append(mean_auc)

        if trial%10 == 0: print("---" + str(trial) + ": cindex", cindex, " mean_auc:", mean_auc)

In [ ]:
getmeanAndCI(cindex_list, name="C-Index")
getmeanAndCI(cindex_list_GB, name="C-Index")
_,p = st.ttest_ind(cindex_list, cindex_list_GB)
print("acc %.3f" % p)
print()

getmeanAndCI(mAUC_list, name="Mean-AUC")
getmeanAndCI(mAUC_list_GB, name="Mean-AUC")
_,p = st.ttest_ind(mAUC_list, mAUC_list_GB)
print("acc %.3f" % p)
print()

In [ ]:
mAUC_list = [x for x in mAUC_list if math.isnan(x) == False]
if DEMOGRAPHCIS:
    cindex_mu_GB_demo, cindex_ci_GB_demo = getmeanAndCI(cindex_list, name="C-Index")
    mAUC_mu_GB_demo, mAUC_ci_GB_demo = getmeanAndCI(mAUC_list, name="Mean-AUC")
    cindex_list_GB_demo = cindex_list
    mAUC_list_GB_demo = mAUC_list
elif MEDICAL:
    cindex_mu_GB_med, cindex_ci_GB_med = getmeanAndCI(cindex_list, name="C-Index")
    mAUC_mu_GB_med, mAUC_ci_GB_med = getmeanAndCI(mAUC_list, name="Mean-AUC")
    cindex_list_GB_med = cindex_list
    mAUC_list_GB_med = mAUC_list
else:
    cindex_mu_GB, cindex_ci_GB = getmeanAndCI(cindex_list, name="C-Index")
    mAUC_mu_GB, mAUC_ci_GB = getmeanAndCI(mAUC_list, name="Mean-AUC")
    cindex_list_GB = cindex_list
    mAUC_list_GB = mAUC_list

### CoxnetSurvivalAnalysis

In [ ]:
if True:
    model = CoxnetSurvivalAnalysis()

    param_grid = {'l1_ratio': [0.01, 0.02, 0.03, 0.04, 0.05, 0.07, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6]
                 }
    cv = RepeatedStratifiedKFold(n_splits=nFold, n_repeats=1, random_state=42)
    gsearch = GridSearchCV(model, param_grid, scoring=score_survival_model,cv=cv)
    gsearch = gsearch.fit(X_mgh, y_mgh)

    print(gsearch.best_params_, gsearch.best_score_)
    gsearch_Coxnet = gsearch
    

In [ ]:
cindex_list = []
mAUC_list = []
for trial in range(nTrials):
    # 5-cross validation
    cv = RepeatedStratifiedKFold(n_splits=nFold, n_repeats=1)
    for train_index, test_index in cv.split(X_mgh, y_mgh):
        X_train, X_val = X_mgh[train_index], X_mgh[test_index]
        y_train, y_val = y_mgh[train_index], y_mgh[test_index]
        
        model = CoxnetSurvivalAnalysis(l1_ratio=gsearch_Coxnet.best_params_['l1_ratio'],
                                       fit_baseline_model=True)
        
        model.fit(X_train, y_train)
        
        cindex = model.score(X_val, y_val)
        cindex_list.append(cindex)
        
        chf_funcs = model.predict_cumulative_hazard_function(X_val)
        
        times_val = list(chf_funcs[0].x)
        times_val.sort()
        times_test = [t for e,t in y_val]
        times_test.sort()
        times =[]
        for t in times_val:
            if t >= times_test[0] and t < times_test[-1]:
                times.append(t)

        risk_scores = np.row_stack([chf(times) for chf in chf_funcs])
        aucs, mean_auc = cumulative_dynamic_auc(y_train, y_val, risk_scores, times)
        mAUC_list.append(mean_auc)

        if trial%10 == 0: print("---" + str(trial) + ": cindex", cindex, " mean_auc:", mean_auc)

In [ ]:
mAUC_list = [x for x in mAUC_list if math.isnan(x) == False]
if DEMOGRAPHCIS:
    cindex_mu_Coxnet_demo, cindex_ci_Coxnet_demo = getmeanAndCI(cindex_list, name="C-Index")
    mAUC_mu_Coxnet_demo, mAUC_ci_Coxnet_demo = getmeanAndCI(mAUC_list, name="Mean-AUC")
    cindex_list_Coxnet_demo = cindex_list
    mAUC_list_Coxnet_demo = mAUC_list
elif MEDICAL:
    cindex_mu_Coxnet_med, cindex_ci_Coxnet_med = getmeanAndCI(cindex_list, name="C-Index")
    mAUC_mu_Coxnet_med, mAUC_ci_Coxnet_med = getmeanAndCI(mAUC_list, name="Mean-AUC")
    cindex_list_Coxnet_med = cindex_list
    mAUC_list_Coxnet_med = mAUC_list
else:
    cindex_mu_Coxnet, cindex_ci_Coxnet = getmeanAndCI(cindex_list, name="C-Index")
    mAUC_mu_Coxnet, mAUC_ci_Coxnet = getmeanAndCI(mAUC_list, name="Mean-AUC")
    cindex_list_Coxnet = cindex_list
    mAUC_list_Coxnet = mAUC_list

### CoxPHSurvivalAnalysis

In [ ]:
if True:
    model = CoxPHSurvivalAnalysis()

    param_grid = {'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                  'n_iter': [70, 100, 150]
                 }
    
    cv = RepeatedStratifiedKFold(n_splits=nFold, n_repeats=1, random_state=42)
    gsearch = GridSearchCV(model, param_grid, scoring=score_survival_model,cv=cv)
    gsearch = gsearch.fit(X_mgh, y_mgh)

    print(gsearch.best_params_, gsearch.best_score_)
    gsearch_CoxPH = gsearch
    

In [ ]:
cindex_list = []
mAUC_list = []
for trial in range(nTrials):
    # 5-cross validation
    cv = RepeatedStratifiedKFold(n_splits=nFold, n_repeats=1)
    for train_index, test_index in cv.split(X_mgh, y_mgh):
        X_train, X_val = X_mgh[train_index], X_mgh[test_index]
        y_train, y_val = y_mgh[train_index], y_mgh[test_index]
        
        model = CoxPHSurvivalAnalysis(alpha=gsearch_CoxPH.best_params_['alpha'],
                                      n_iter=gsearch_CoxPH.best_params_['n_iter']
                                     )
        
        model.fit(X_train, y_train)
        
        cindex = model.score(X_val, y_val)
        cindex_list.append(cindex)
        
        chf_funcs = model.predict_cumulative_hazard_function(X_val)
        
        times_val = list(chf_funcs[0].x)
        times_val.sort()
        times_test = [t for e,t in y_val]
        times_test.sort()
        times =[]
        for t in times_val:
            if t >= times_test[0] and t < times_test[-1]:
                times.append(t)

        risk_scores = np.row_stack([chf(times) for chf in chf_funcs])
        aucs, mean_auc = cumulative_dynamic_auc(y_train, y_val, risk_scores, times)
        mAUC_list.append(mean_auc)

        if trial%10 == 0: print("---" + str(trial) + ": cindex", cindex, " mean_auc:", mean_auc)

In [ ]:
mAUC_list = [x for x in mAUC_list if math.isnan(x) == False]
if DEMOGRAPHCIS:
    cindex_mu_CoxPH_demo, cindex_ci_CoxPH_demo = getmeanAndCI(cindex_list, name="C-Index")
    mAUC_mu_CoxPH_demo, mAUC_ci_CoxPH_demo = getmeanAndCI(mAUC_list, name="Mean-AUC")
    cindex_list_CoxPH_demo = cindex_list
    mAUC_list_CoxPH_demo = mAUC_list
elif MEDICAL:
    cindex_mu_CoxPH_med, cindex_ci_CoxPH_med = getmeanAndCI(cindex_list, name="C-Index")
    mAUC_mu_CoxPH_med, mAUC_ci_CoxPH_med = getmeanAndCI(mAUC_list, name="Mean-AUC")
    cindex_list_CoxPH_med = cindex_list
    mAUC_list_CoxPH_med = mAUC_list
else:
    cindex_mu_CoxPH, cindex_ci_CoxPH = getmeanAndCI(cindex_list, name="C-Index")
    mAUC_mu_CoxPH, mAUC_ci_CoxPH = getmeanAndCI(mAUC_list, name="Mean-AUC")
    cindex_list_CoxPH = cindex_list
    mAUC_list_CoxPH = mAUC_list

## 3. External validation

In [ ]:
model = RandomSurvivalForest(n_estimators=gsearch_RF.best_params_['n_estimators'],
                              min_samples_split=gsearch_RF.best_params_['min_samples_split'],
                              min_samples_leaf=gsearch_RF.best_params_['min_samples_leaf'])

cindex_list = []
mAUC_list = []
for trial in range(nTrials):
    X_train = X_mgh
    y_train = y_mgh
    
    # use 95% in the mgh cohort for randomness
    X_train, _, y_train, _ = train_test_split(X_mgh, y_mgh, test_size = 0.05, shuffle = True)
    
    X_val = X_dfci
    y_val = y_dfci
    
    model.fit(X_train, y_train)
    
    cindex = model.score(X_val, y_val)
    cindex_list.append(cindex)
    
    chf_funcs = model.predict_cumulative_hazard_function(X_val, return_array=False)
    
    times_val = list(chf_funcs[0].x)
    times_val.sort()
    times_test = [t for e,t in y_val]
    times_test.sort()
    times =[]
    for t in times_val:
        if t >= times_test[0] and t < times_test[-1]:
            times.append(t)
    
    risk_scores = np.row_stack([chf(times) for chf in chf_funcs])
    aucs, mean_auc = cumulative_dynamic_auc(y_train, y_val, risk_scores, times)
    mAUC_list.append(mean_auc)
    
    if trial%10 == 0: print("---" + str(trial) + ": cindex", cindex, " mean_auc:", mean_auc)

In [ ]:
getmeanAndCI(cindex_list, name="C-Index")
getmeanAndCI(cindex_list_RF_dfci, name="C-Index")
_,p = st.ttest_ind(cindex_list, cindex_list_RF_dfci)
print("acc %.3f" % p)
print()

getmeanAndCI(mAUC_list, name="Mean-AUC")
getmeanAndCI(mAUC_list_RF_dfci, name="Mean-AUC")
_,p = st.ttest_ind(mAUC_list, mAUC_list_RF_dfci)
print("acc %.3f" % p)
print()

In [ ]:
if DEMOGRAPHCIS:
    cindex_mu_RF_dfci_demo, cindex_ci_RF_dfci_demo = getmeanAndCI(cindex_list, name="C-Index")
    mAUC_mu_RF_dfci_demo, mAUC_ci_RF_dfci_demo = getmeanAndCI(mAUC_list, name="Mean-AUC")
    cindex_list_RF_dfci_demo = cindex_list
    mAUC_list_RF_dfci_demo = mAUC_list
elif MEDICAL:
    cindex_mu_RF_dfci_med, cindex_ci_RF_dfci_med = getmeanAndCI(cindex_list, name="C-Index")
    mAUC_mu_RF_dfci_med, mAUC_ci_RF_dfci_med = getmeanAndCI(mAUC_list, name="Mean-AUC")
    cindex_list_RF_dfci_med = cindex_list
    mAUC_list_RF_dfci_med = mAUC_list
else:
    cindex_mu_RF_dfci, cindex_ci_RF_dfci = getmeanAndCI(cindex_list, name="C-Index")
    mAUC_mu_RF_dfci, mAUC_ci_RF_dfci = getmeanAndCI(mAUC_list, name="Mean-AUC")
    cindex_list_RF_dfci = cindex_list
    mAUC_list_RF_dfci = mAUC_list

In [ ]:
model = GradientBoostingSurvivalAnalysis(
            loss = 'coxph', 
            n_estimators=gsearch_GB.best_params_['n_estimators'], 
            learning_rate=gsearch_GB.best_params_['learning_rate'], 
            max_depth=gsearch_GB.best_params_['max_depth']
        )

cindex_list = []
mAUC_list = []
for trial in range(nTrials):
    X_train = X_mgh
    y_train = y_mgh
    
    X_train, X_val, y_train, y_val = train_test_split(X_mgh, y_mgh, test_size = 0.05, shuffle = True)
    
    X_val = X_dfci
    y_val = y_dfci
    
    
    model.fit(X_train, y_train)
    
    cindex = model.score(X_val, y_val)
    cindex_list.append(cindex)
    
    chf_funcs = model.predict_cumulative_hazard_function(X_val)
    
    times_val = list(chf_funcs[0].x)
    times_val.sort()
    times_test = [t for e,t in y_val]
    times_test.sort()
    times =[]
    for t in times_val:
        if t >= times_test[0] and t < times_test[-1]:
            times.append(t)
    
    risk_scores = np.row_stack([chf(times) for chf in chf_funcs])
    aucs, mean_auc = cumulative_dynamic_auc(y_train, y_val, risk_scores, times)
    mAUC_list.append(mean_auc)
    
    if trial%10 == 0: print("---" + str(trial) + ": cindex", cindex, " mean_auc:", mean_auc)

In [ ]:
getmeanAndCI(cindex_list, name="C-Index")
getmeanAndCI(cindex_list_GB_dfci, name="C-Index")
_,p = st.ttest_ind(cindex_list, cindex_list_GB_dfci)
print("acc %.3f" % p)
print()

getmeanAndCI(mAUC_list, name="Mean-AUC")
getmeanAndCI(mAUC_list_GB_dfci, name="Mean-AUC")
_,p = st.ttest_ind(mAUC_list, mAUC_list_GB_dfci)
print("acc %.3f" % p)
print()

cindex_GB_noT_noS_ex = cindex_list
mAUC_GB_noT_noS_ex = mAUC_list

In [ ]:
if DEMOGRAPHCIS:
    cindex_mu_GB_dfci_demo, cindex_ci_GB_dfci_demo = getmeanAndCI(cindex_list, name="C-Index")
    mAUC_mu_GB_dfci_demo, mAUC_ci_GB_dfci_demo = getmeanAndCI(mAUC_list, name="Mean-AUC")
    cindex_list_GB_dfci_demo = cindex_list
    mAUC_list_GB_dfci_demo = mAUC_list
elif MEDICAL:
    cindex_mu_GB_dfci_med, cindex_ci_GB_dfci_med = getmeanAndCI(cindex_list, name="C-Index")
    mAUC_mu_GB_dfci_med, mAUC_ci_GB_dfci_med = getmeanAndCI(mAUC_list, name="Mean-AUC")
    cindex_list_GB_dfci_med = cindex_list
    mAUC_list_GB_dfci_med = mAUC_list
else:
    cindex_mu_GB_dfci, cindex_ci_GB_dfci = getmeanAndCI(cindex_list, name="C-Index")
    mAUC_mu_GB_dfci, mAUC_ci_GB_dfci = getmeanAndCI(mAUC_list, name="Mean-AUC")
    cindex_list_GB_dfci = cindex_list
    mAUC_list_GB_dfci = mAUC_list

In [ ]:
model = CoxnetSurvivalAnalysis(l1_ratio=gsearch_Coxnet.best_params_['l1_ratio'],
                                       fit_baseline_model=True)

cindex_list = []
mAUC_list = []
for trial in range(nTrials):
    X_train = X_mgh
    y_train = y_mgh
    
    X_train, X_val, y_train, y_val = train_test_split(X_mgh, y_mgh, test_size = 0.05, shuffle = True)
    
    X_val = X_dfci
    y_val = y_dfci
    
    model.fit(X_train, y_train)
    
    cindex = model.score(X_val, y_val)
    cindex_list.append(cindex)
    
    chf_funcs = model.predict_cumulative_hazard_function(X_val)
    
    times_val = list(chf_funcs[0].x)
    times_val.sort()
    times_test = [t for e,t in y_val]
    times_test.sort()
    times =[]
    for t in times_val:
        if t >= times_test[0] and t < times_test[-1]:
            times.append(t)
    
    risk_scores = np.row_stack([chf(times) for chf in chf_funcs])
    aucs, mean_auc = cumulative_dynamic_auc(y_train, y_val, risk_scores, times)
    mAUC_list.append(mean_auc)
    
    if trial%10 == 0: print("---" + str(trial) + ": cindex", cindex, " mean_auc:", mean_auc)

In [ ]:
if DEMOGRAPHCIS:
    cindex_mu_Coxnet_dfci_demo, cindex_ci_Coxnet_dfci_demo = getmeanAndCI(cindex_list, name="C-Index")
    mAUC_mu_Coxnet_dfci_demo, mAUC_ci_Coxnet_dfci_demo = getmeanAndCI(mAUC_list, name="Mean-AUC")
    cindex_list_Coxnet_dfci_demo = cindex_list
    mAUC_list_Coxnet_dfci_demo = mAUC_list
elif MEDICAL:
    cindex_mu_Coxnet_dfci_med, cindex_ci_Coxnet_dfci_med = getmeanAndCI(cindex_list, name="C-Index")
    mAUC_mu_Coxnet_dfci_med, mAUC_ci_Coxnet_dfci_med = getmeanAndCI(mAUC_list, name="Mean-AUC")
    cindex_list_Coxnet_med = cindex_list
    mAUC_list_Coxnet_med = mAUC_list
else:
    cindex_mu_Coxnet_dfci, cindex_ci_Coxnet_dfci = getmeanAndCI(cindex_list, name="C-Index")
    mAUC_mu_Coxnet_dfci, mAUC_ci_Coxnet_dfci = getmeanAndCI(mAUC_list, name="Mean-AUC")
    cindex_list_Coxnet_dfci = cindex_list
    mAUC_list_Coxnet_dfci = mAUC_list

In [ ]:
model = CoxPHSurvivalAnalysis(alpha=gsearch_CoxPH.best_params_['alpha'],
                                      n_iter=gsearch_CoxPH.best_params_['n_iter']
                                     )

cindex_list = []
mAUC_list = []
for trial in range(nTrials):
    X_train = X_mgh
    y_train = y_mgh
    
    X_train, X_val, y_train, y_val = train_test_split(X_mgh, y_mgh, test_size = 0.05, shuffle = True)
    
    X_val = X_dfci
    y_val = y_dfci
    
    model.fit(X_train, y_train)
    
    cindex = model.score(X_val, y_val)
    cindex_list.append(cindex)
    
    chf_funcs = model.predict_cumulative_hazard_function(X_val)
    
    times_val = list(chf_funcs[0].x)
    times_val.sort()
    times_test = [t for e,t in y_val]
    times_test.sort()
    times =[]
    for t in times_val:
        if t >= times_test[0] and t < times_test[-1]:
            times.append(t)
    
    risk_scores = np.row_stack([chf(times) for chf in chf_funcs])
    aucs, mean_auc = cumulative_dynamic_auc(y_train, y_val, risk_scores, times)
    mAUC_list.append(mean_auc)
    
    if trial%10 == 0: print("---" + str(trial) + ": cindex", cindex, " mean_auc:", mean_auc)

In [ ]:
if DEMOGRAPHCIS:
    cindex_mu_CoxPH_dfci_demo, cindex_ci_CoxPH_dfci_demo = getmeanAndCI(cindex_list, name="C-Index")
    mAUC_mu_CoxPH_dfci_demo, mAUC_ci_CoxPH_dfci_demo = getmeanAndCI(mAUC_list, name="Mean-AUC")
    cindex_list_CoxPH_dfci_demo = cindex_list
    mAUC_list_CoxPH_dfci_demo = mAUC_list
elif MEDICAL:
    cindex_mu_CoxPH_dfci_med, cindex_ci_CoxPH_dfci_med = getmeanAndCI(cindex_list, name="C-Index")
    mAUC_mu_CoxPH_dfci_med, mAUC_ci_CoxPH_dfci_med = getmeanAndCI(mAUC_list, name="Mean-AUC")
    cindex_list_CoxPH_dfci_med = cindex_list
    mAUC_list_CoxPH_dfci_med = mAUC_list
else:
    cindex_mu_CoxPH_dfci, cindex_ci_CoxPH_dfci = getmeanAndCI(cindex_list, name="C-Index")
    mAUC_mu_CoxPH_dfci, mAUC_ci_CoxPH_dfci = getmeanAndCI(mAUC_list, name="Mean-AUC")
    cindex_list_CoxPH_dfci = cindex_list
    mAUC_list_CoxPH_dfci = mAUC_list

In [ ]:
mu, ci = getmeanAndCI(mAUC_list_GB, "GB", True)
mu, ci = getmeanAndCI(mAUC_list_RF, "RF", True)
_,p = st.ttest_ind(mAUC_list_GB, mAUC_list_RF)
print("p %.3f" % p)

In [ ]:
mu, ci = getmeanAndCI(mAUC_list_GB_dfci, "GB", True)
mu, ci = getmeanAndCI(mAUC_list_RF_dfci, "RF", True)
_,p = st.ttest_ind(mAUC_list_GB_dfci, mAUC_list_RF_dfci)
print("p %.3f" % p)

## 4. Visualization

In [ ]:
coxnet = "Coxnet"
coxph = "CoxPH"
rf_t = "RF-T"
gb_t = "GB-T"

### Internal validation with different features

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(15,2), sharey=False, dpi=1024)

y = 0
ax1.plot((mAUC_ci_CoxPH_demo[0],mAUC_ci_CoxPH_demo[1]),(y,y),'ro--',color='gray')
ax1.plot(mAUC_mu_CoxPH_demo, y, 'o', color='black')

ax1.plot((mAUC_ci_CoxPH_med[0],mAUC_ci_CoxPH_med[1]),(y,y),'ro--',color='blue')
ax1.plot(mAUC_mu_CoxPH_med, y, 'o', color='green')

ax1.plot((mAUC_ci_CoxPH[0],mAUC_ci_CoxPH[1]),(y,y),'ro--',color='orange')
ax1.plot(mAUC_mu_CoxPH, y, 'o', color='#f44336')

# 
ax2.plot((cindex_ci_CoxPH_demo[0],cindex_ci_CoxPH_demo[1]),(y,y),'ro--',color='gray')
ax2.plot(cindex_mu_CoxPH_demo, y, 'o', color='black')

ax2.plot((cindex_ci_CoxPH_med[0],cindex_ci_CoxPH_med[1]),(y,y),'ro--',color='blue')
ax2.plot(cindex_mu_CoxPH_med, y, 'o', color='green')

ax2.plot((cindex_ci_CoxPH[0],cindex_ci_CoxPH[1]),(y,y),'ro--',color='orange')
ax2.plot(cindex_mu_CoxPH, y, 'o', color='#f44336')

y += 1

ax1.plot((mAUC_ci_Coxnet_demo[0],mAUC_ci_Coxnet_demo[1]),(y,y),'ro--',color='gray')
ax1.plot(mAUC_mu_Coxnet_demo, y, 'o', color='black')

ax1.plot((mAUC_ci_Coxnet_med[0],mAUC_ci_Coxnet_med[1]),(y,y),'ro--',color='blue')
ax1.plot(mAUC_mu_Coxnet_med, y, 'o', color='green')

ax1.plot((mAUC_ci_Coxnet[0],mAUC_ci_Coxnet[1]),(y,y),'ro--',color='orange')
ax1.plot(mAUC_mu_Coxnet, y, 'o', color='#f44336')

#
ax2.plot((cindex_ci_Coxnet_demo[0],cindex_ci_Coxnet_demo[1]),(y,y),'ro--',color='gray')
ax2.plot(cindex_mu_Coxnet_demo, y, 'o', color='black')

ax2.plot((cindex_ci_Coxnet_med[0],cindex_ci_Coxnet_med[1]),(y,y),'ro--',color='blue')
ax2.plot(cindex_mu_Coxnet_med, y, 'o', color='green')

ax2.plot((cindex_ci_Coxnet[0],cindex_ci_Coxnet[1]),(y,y),'ro--',color='orange')
ax2.plot(cindex_mu_Coxnet, y, 'o', color='#f44336')

y += 1
ax1.plot((mAUC_ci_GB_demo[0],mAUC_ci_GB_demo[1]),(y,y),'ro--',color='gray')
ax1.plot(mAUC_mu_GB_demo, y, 'o', color='black')

ax1.plot((mAUC_ci_GB_med[0],mAUC_ci_GB_med[1]),(y,y),'ro--',color='blue')
ax1.plot(mAUC_mu_GB_med, y, 'o', color='green')

ax1.plot((mAUC_ci_GB[0],mAUC_ci_GB[1]),(y,y),'ro--',color='orange')
ax1.plot(mAUC_mu_GB, y, 'o', color='#f44336')

#
ax2.plot((cindex_ci_GB_demo[0],cindex_ci_GB_demo[1]),(y,y),'ro--',color='gray')
ax2.plot(cindex_mu_GB_demo, y, 'o', color='black')

ax2.plot((cindex_ci_GB_med[0],cindex_ci_GB_med[1]),(y,y),'ro--',color='blue')
ax2.plot(cindex_mu_GB_med, y, 'o', color='green')

ax2.plot((cindex_ci_GB[0],cindex_ci_GB[1]),(y,y),'ro--',color='orange')
ax2.plot(cindex_mu_GB, y, 'o', color='#f44336')


y += 1

ax1.plot((mAUC_ci_RF_demo[0],mAUC_ci_RF_demo[1]),(y,y),'ro--',color='gray', label="demographics only")
ax1.plot(mAUC_mu_RF_demo, y, 'o--', color='black')

ax1.plot((mAUC_ci_RF_med[0],mAUC_ci_RF_med[1]),(y,y),'ro--',color='blue', label="demographics + medical history")
ax1.plot(mAUC_mu_RF_med, y, 'o--', color='green')

ax1.plot((mAUC_ci_RF[0],mAUC_ci_RF[1]),(y,y),'ro--',color='orange', label="all extracted features")
ax1.plot(mAUC_mu_RF, y, 'o--', color='#f44336')

#
ax2.plot((cindex_ci_RF_demo[0],cindex_ci_RF_demo[1]),(y,y),'ro--',color='gray', label="demographics")
ax2.plot(cindex_mu_RF_demo, y, 'o--', color='black')

ax2.plot((cindex_ci_RF_med[0],cindex_ci_RF_med[1]),(y,y),'ro--',color='blue', label="medical history")
ax2.plot(cindex_mu_RF_med, y, 'o--', color='green')

ax2.plot((cindex_ci_RF[0],cindex_ci_RF[1]),(y,y),'ro--',color='orange', label="tumor features")
ax2.plot(cindex_mu_RF, y, 'o--', color='#f44336')


y += 1
ax1.set_yticks([0, 1, 2, 3])
ax1.set_yticklabels([coxph, coxnet, gb_t, rf_t])

ax2.set_yticks([0, 1, 2, 3])
ax2.set_yticklabels([coxph, coxnet, gb_t, rf_t])

ax1.legend(bbox_to_anchor=(0, 1.56), loc='upper left')

ax1.set_xlabel("time-dependent AUC: mean and 95% confidence interval\n(internal validation)")
ax2.set_xlabel("concordance index: mean and 95% confidence interval\n(internal validation)")

ax1.set_xlim(0.53, 0.87)
ax2.set_xlim(0.53, 0.855)

### Cross-institutional and different features

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(15,2), sharey=False, dpi=1024)

y = 0
ax1.plot((mAUC_ci_CoxPH_dfci_demo[0],mAUC_ci_CoxPH_dfci_demo[1]),(y,y),'ro--',color='gray')
ax1.plot(mAUC_mu_CoxPH_dfci_demo, y, 'o', color='black')

ax1.plot((mAUC_ci_CoxPH_dfci_med[0],mAUC_ci_CoxPH_dfci_med[1]),(y,y),'ro--',color='blue')
ax1.plot(mAUC_mu_CoxPH_dfci_med, y, 'o', color='green')

ax1.plot((mAUC_ci_CoxPH_dfci[0],mAUC_ci_CoxPH_dfci[1]),(y,y),'ro--',color='orange')
ax1.plot(mAUC_mu_CoxPH_dfci, y, 'o', color='#f44336')

#

ax2.plot((cindex_ci_CoxPH_dfci_demo[0],cindex_ci_CoxPH_dfci_demo[1]),(y,y),'ro--',color='gray')
ax2.plot(cindex_mu_CoxPH_dfci_demo, y, 'o', color='black')

ax2.plot((cindex_ci_CoxPH_dfci_med[0],cindex_ci_CoxPH_dfci_med[1]),(y,y),'ro--',color='blue')
ax2.plot(cindex_mu_CoxPH_dfci_med, y, 'o', color='green')

ax2.plot((cindex_ci_CoxPH_dfci[0],cindex_ci_CoxPH_dfci[1]),(y,y),'ro--',color='orange')
ax2.plot(cindex_mu_CoxPH_dfci, y, 'o', color='#f44336')


y += 1
ax1.plot((mAUC_ci_Coxnet_dfci_demo[0],mAUC_ci_Coxnet_dfci_demo[1]),(y,y),'ro--',color='gray')
ax1.plot(mAUC_mu_Coxnet_dfci_demo, y, 'o', color='black')

ax1.plot((mAUC_ci_Coxnet_dfci_med[0],mAUC_ci_Coxnet_dfci_med[1]),(y,y),'ro--',color='blue')
ax1.plot(mAUC_mu_Coxnet_dfci_med, y, 'o', color='green')

ax1.plot((mAUC_ci_Coxnet_dfci[0],mAUC_ci_Coxnet_dfci[1]),(y,y),'ro--',color='orange')
ax1.plot(mAUC_mu_Coxnet_dfci, y, 'o', color='#f44336')

ax2.plot((cindex_ci_Coxnet_dfci_demo[0],cindex_ci_Coxnet_dfci_demo[1]),(y,y),'ro--',color='gray')
ax2.plot(cindex_mu_Coxnet_dfci_demo, y, 'o', color='black')

ax2.plot((cindex_ci_Coxnet_dfci_med[0],cindex_ci_Coxnet_dfci_med[1]),(y,y),'ro--',color='blue')
ax2.plot(cindex_mu_Coxnet_dfci_med, y, 'o', color='green')

ax2.plot((cindex_ci_Coxnet_dfci[0],cindex_ci_Coxnet_dfci[1]),(y,y),'ro--',color='orange')
ax2.plot(cindex_mu_Coxnet_dfci, y, 'o', color='#f44336')


y += 1
ax1.plot((mAUC_ci_GB_dfci_demo[0],mAUC_ci_GB_dfci_demo[1]),(y,y),'ro--',color='gray')
ax1.plot(mAUC_mu_GB_dfci_demo, y, 'o', color='black')

ax1.plot((mAUC_ci_GB_dfci_med[0],mAUC_ci_GB_dfci_med[1]),(y,y),'ro--',color='blue')
ax1.plot(mAUC_mu_GB_dfci_med, y, 'o', color='green')

ax1.plot((mAUC_ci_GB_dfci[0],mAUC_ci_GB_dfci[1]),(y,y),'ro--',color='orange')
ax1.plot(mAUC_mu_GB_dfci, y, 'o', color='#f44336')



ax2.plot((cindex_ci_GB_dfci_demo[0],cindex_ci_GB_dfci_demo[1]),(y,y),'ro--',color='gray')
ax2.plot(cindex_mu_GB_dfci_demo, y, 'o', color='black')

ax2.plot((cindex_ci_GB_dfci_med[0],cindex_ci_GB_dfci_med[1]),(y,y),'ro--',color='blue')
ax2.plot(cindex_mu_GB_dfci_med, y, 'o', color='green')

ax2.plot((cindex_ci_GB_dfci[0],cindex_ci_GB_dfci[1]),(y,y),'ro--',color='orange')
ax2.plot(cindex_mu_GB_dfci, y, 'o', color='#f44336')



y += 1

ax1.plot((mAUC_ci_RF_dfci_demo[0],mAUC_ci_RF_dfci_demo[1]),(y,y),'ro--',color='gray', label="demographics only")
ax1.plot(mAUC_mu_RF_dfci_demo, y, 'o--', color='black')

ax1.plot((mAUC_ci_RF_dfci_med[0],mAUC_ci_RF_dfci_med[1]),(y,y),'ro--',color='blue', label="demographics + medical history")
ax1.plot(mAUC_mu_RF_dfci_med, y, 'o--', color='green')

ax1.plot((mAUC_ci_RF_dfci[0],mAUC_ci_RF_dfci[1]),(y,y),'ro--',color='orange', label="all extracted features")
ax1.plot(mAUC_mu_RF_dfci, y, 'o--', color='#f44336')

ax2.plot((cindex_ci_RF_dfci_demo[0],cindex_ci_RF_dfci_demo[1]),(y,y),'ro--',color='gray', label="demographics only")
ax2.plot(cindex_mu_RF_dfci_demo, y, 'o--', color='black')

ax2.plot((cindex_ci_RF_dfci_med[0],cindex_ci_RF_dfci_med[1]),(y,y),'ro--',color='blue', label="demographics + medical history")
ax2.plot(cindex_mu_RF_dfci_med, y, 'o--', color='green')

ax2.plot((cindex_ci_RF_dfci[0],cindex_ci_RF_dfci[1]),(y,y),'ro--',color='orange', label="all extracted features")
ax2.plot(cindex_mu_RF_dfci, y, 'o--', color='#f44336')


y += 1

# ax1.legend(bbox_to_anchor=(0, 1.56), loc='upper left')

ax1.set_yticks([0, 1, 2, 3])
ax1.set_yticklabels([coxph, coxnet, gb_t, rf_t])

ax2.set_yticks([0, 1, 2, 3])
ax2.set_yticklabels([coxph, coxnet, gb_t, rf_t])

ax1.set_xlabel("time-dependent AUC: mean and 95% confidence interval\n(external validation)")
ax2.set_xlabel("concordance index: mean and 95% confidence interval\n(external validation)")

ax1.set_xlim(0.53, 0.855)
ax2.set_xlim(0.53, 0.855)

## internal validation and cross-institutional validation

In [ ]:
print("-----RF")
print(f"AUC-T:{round(mAUC_mu_RF, 3)} CI:{round(mAUC_ci_RF[0], 3)}-{round(mAUC_ci_RF[1], 3)}")
print(f"C-Index:{round(cindex_mu_RF, 3)} CI:{round(cindex_ci_RF[0], 3)}-{round(cindex_ci_RF[1], 3)}")

print()
print(f"AUC-T:{round(mAUC_mu_RF_dfci, 3)} CI:{round(mAUC_ci_RF_dfci[0], 3)}-{round(mAUC_ci_RF_dfci[1], 3)}")
print(f"C-Index:{round(cindex_mu_RF_dfci, 3)} CI:{round(cindex_ci_RF_dfci[0], 3)}-{round(cindex_ci_RF_dfci[1], 3)}")
_,p1 = st.ttest_ind(mAUC_list_RF, mAUC_list_RF_dfci)
_,p2 = st.ttest_ind(cindex_list_RF, cindex_list_RF_dfci)
print("auc p %.3f" % p1)
print("cindex %.3f" % p2)

print("-----GB")
print(f"AUC-T:{round(mAUC_mu_GB, 3)} CI:{round(mAUC_ci_GB[0], 3)}-{round(mAUC_ci_GB[1], 3)}")
print(f"C-Index:{round(cindex_mu_GB, 3)} CI:{round(cindex_ci_GB[0], 3)}-{round(cindex_ci_GB[1], 3)}")
print()
print(f"AUC-T:{round(mAUC_mu_GB_dfci, 3)} CI:{round(mAUC_ci_GB_dfci[0], 3)}-{round(mAUC_ci_GB_dfci[1], 3)}")
print(f"C-Index:{round(cindex_mu_GB_dfci, 3)} CI:{round(cindex_ci_GB_dfci[0], 3)}-{round(cindex_ci_GB_dfci[1], 3)}")
_,p1 = st.ttest_ind(mAUC_list_GB, mAUC_list_GB_dfci)
_,p2 = st.ttest_ind(cindex_list_GB, cindex_list_GB_dfci)
print("auc p %.3f" % p1)
print("cindex %.3f" % p2)

print("-----Coxnet")
print(f"AUC-T:{round(mAUC_mu_Coxnet, 3)} CI:{round(mAUC_ci_Coxnet[0], 3)}-{round(mAUC_ci_Coxnet[1], 3)}")
print(f"C-Index:{round(cindex_mu_Coxnet, 3)} CI:{round(cindex_ci_Coxnet[0], 3)}-{round(cindex_ci_Coxnet[1], 3)}")
print()
print(f"AUC-T:{round(mAUC_mu_Coxnet_dfci, 3)} CI:{round(mAUC_ci_Coxnet_dfci[0], 3)}-{round(mAUC_ci_Coxnet_dfci[1], 3)}")
print(f"C-Index:{round(cindex_mu_Coxnet_dfci, 3)} CI:{round(cindex_ci_Coxnet_dfci[0], 3)}-{round(cindex_ci_Coxnet_dfci[1], 3)}")
_,p1 = st.ttest_ind(mAUC_list_Coxnet, mAUC_list_Coxnet_dfci)
_,p2 = st.ttest_ind(cindex_list_Coxnet, cindex_list_Coxnet_dfci)
print("auc p %.3f" % p1)
print("cindex %.3f" % p2)

print("-----CoxPH")
print(f"AUC-T:{round(mAUC_mu_CoxPH, 3)} CI:{round(mAUC_ci_CoxPH[0], 3)}-{round(mAUC_ci_CoxPH[1], 3)}")
print(f"C-Index:{round(cindex_mu_CoxPH, 3)} CI:{round(cindex_ci_CoxPH[0], 3)}-{round(cindex_ci_CoxPH[1], 3)}")
print()
print(f"AUC-T:{round(mAUC_mu_CoxPH_dfci, 3)} CI:{round(mAUC_ci_CoxPH_dfci[0], 3)}-{round(mAUC_ci_CoxPH_dfci[1], 3)}")
print(f"C-Index:{round(cindex_mu_CoxPH_dfci, 3)} CI:{round(cindex_ci_CoxPH_dfci[0], 3)}-{round(cindex_ci_CoxPH_dfci[1], 3)}")
_,p1 = st.ttest_ind(mAUC_list_CoxPH, mAUC_list_CoxPH_dfci)
_,p2 = st.ttest_ind(cindex_list_CoxPH, cindex_list_CoxPH_dfci)
print("auc p %.3f" % p1)
print("cindex %.3f" % p2)


In [ ]:
getmeanAndCI(mAUC_list_GB, 'GB')
getmeanAndCI(mAUC_list_RF, 'RF')
_,p1 = st.ttest_ind(mAUC_list_RF, mAUC_list_GB)
print("auc p %.3f" % p1)
print()
getmeanAndCI(cindex_list_GB, 'GB')
getmeanAndCI(cindex_list_RF, 'RF')
_,p2 = st.ttest_ind(cindex_list_RF, cindex_list_GB)
print("cindex %.3f" % p2)
print("----------")
getmeanAndCI(mAUC_list_GB_dfci, 'GB')
getmeanAndCI(mAUC_list_RF_dfci, 'RF')
_,p1 = st.ttest_ind(mAUC_list_RF_dfci, mAUC_list_GB_dfci)
print("auc p %.3f" % p1)
print()
getmeanAndCI(cindex_list_GB_dfci, 'GB')
getmeanAndCI(cindex_list_RF_dfci, 'RF')
_,p2 = st.ttest_ind(cindex_list_RF_dfci, cindex_list_GB_dfci)
print("cindex %.3f" % p2)

### Example of prediction
#### RF-T

In [ ]:
model = RandomSurvivalForest(n_estimators=gsearch_RF.best_params_['n_estimators'],
                              min_samples_split=gsearch_RF.best_params_['min_samples_split'],
                              min_samples_leaf=gsearch_RF.best_params_['min_samples_leaf'])

model.fit(X_mgh, y_mgh)

In [ ]:
targetX = X_dfci
targety = y_dfci
model.score(targetX, targety)

cnt = 7
selected = random.sample(range(len(targety)), k=cnt)
print(selected)
selected = [ 49, 163, 28, 192, 241, 50, 161] # change the order to avoid red and green together

sample_X = targetX[selected]
sample_y = targety[selected]
# print(sample_y)
surv = model.predict_survival_function(sample_X, return_array=True)

Years = 10

fig, ax = plt.subplots(figsize=(5,2.5),dpi=512)
for i, s in enumerate(surv):
    points = len(model.event_times_[model.event_times_<=Years])
    plt.step(model.event_times_[:points], (1-s)[:points], where="post", label= "ID" + str(i)+" : "+str(sample_y[i]))
plt.ylabel("Recurrence probability")
plt.xlabel("Years from diagnosis\n (RF-T)")

#### GB-T

In [ ]:
print(gsearch_GB.best_params_)
model = GradientBoostingSurvivalAnalysis(
            loss = 'coxph', 
            n_estimators=gsearch_GB.best_params_['n_estimators'], 
            learning_rate=gsearch_GB.best_params_['learning_rate'], 
            max_depth=gsearch_GB.best_params_['max_depth']
        )
model.fit(X_mgh, y_mgh)
model.score(X_dfci, y_dfci)


In [ ]:
sample_X = X_dfci[selected]
sample_y = y_dfci[selected]
surv = model.predict_survival_function(sample_X)


fig, ax = plt.subplots(figsize=(5,2.5),dpi=216)
for i, s in enumerate(surv):
    points = len(s.x[s.x<=Years])
    plt.step(s.x[:points], (1-s.y)[:points], where="post", label= "ID" + str(i)+" : "+str(sample_y[i]))
plt.ylabel("Recurrence probability")
plt.xlabel("Years from diagnosis\n (GB-T)")

#### Coxnet

In [ ]:
print(gsearch_Coxnet.best_params_)
model = CoxnetSurvivalAnalysis(l1_ratio=gsearch_Coxnet.best_params_['l1_ratio'],
                                       fit_baseline_model=True)
model.fit(X_mgh, y_mgh)
model.score(X_dfci, y_dfci)


In [ ]:
sample_X = X_dfci[selected]
sample_y = y_dfci[selected]
surv = model.predict_survival_function(sample_X)


fig, ax = plt.subplots(figsize=(5,2.5),dpi=216)
for i, s in enumerate(surv):
    points = len(s.x[s.x<=Years])
    plt.step(s.x[:points], (1-s.y)[:points], where="post", label= "ID" + str(i)+" : "+str(sample_y[i]))
plt.ylabel("Recurrence probability")
plt.xlabel("Years from diagnosis\n (Coxnet)")


#### CoxPH

In [ ]:
print(gsearch_Coxnet.best_params_)
model = CoxPHSurvivalAnalysis(alpha=gsearch_CoxPH.best_params_['alpha'],
                                      n_iter=gsearch_CoxPH.best_params_['n_iter']
                                     )
model.fit(X_mgh, y_mgh)
model.score(X_dfci, y_dfci)


In [ ]:
sample_X = X_dfci[selected]
sample_y = y_dfci[selected]
surv = model.predict_survival_function(sample_X)


fig, ax = plt.subplots(figsize=(5,2.5),dpi=216)
for i, s in enumerate(surv):
    points = len(s.x[s.x<=Years])
    plt.step(s.x[:points], (1-s.y)[:points], where="post", label= "ID" + str(i)+" : "+str(sample_y[i]))
plt.ylabel("Recurrence probability")
plt.xlabel("Years from diagnosis\n (CoxPH)")

## 5. Feature Importance

In [ ]:
data_MGH = melanoma_data # on all data
data_MGH = melanoma_data[melanoma_data['Site'] == "MGB"]
data_DFCI = melanoma_data[melanoma_data['Site'] == "DFCI"]

data_MGH = data_MGH.drop(columns = ['Site'])
data_DFCI = data_DFCI.drop(columns = ['Site'])

X_mgh, y_mgh = get_x_y(data_MGH, attr_labels=['Recurrence','Dia2Recur'], pos_label=True, survival=True)

X_dfci, y_dfci = get_x_y(data_DFCI, attr_labels=['Recurrence','Dia2Recur'], pos_label=True, survival=True)

scaler = StandardScaler()
X_mgh = scaler.fit_transform(X_mgh)
scaler = StandardScaler()
X_dfci = scaler.fit_transform(X_dfci)


In [ ]:
print(len(data_MGH.columns))

In [ ]:
feature_names = ['Age at diagnosis', 'Median income',
       'CCS', 'HPCM: Yes', 
                 'Thickness', 
                 'Anatomic level',
       'Mitotic rate', 'Total margins', 'Sex: Male', 'Race: White',
       'Ethnicity: Hispanic', 'Ethnicity: Non-Hispanic', 'Ethnicity: Unknown',
       'Insurance type: Medicaid', 'Insurance type: Medicare',
       'Insurance type: Private', 'Insurance type: Self.pay',
       'Marital status: Divorced', 'Marital status: Married',
       'Marital status: Other', 'Marital status: Single',
       'Marital status: Unavailable', 'Marital status: Widowed', 
       'Stage: 1A', 'Stage: 1B', 'Stage: 2A', 'Stage: 2B', 'Stage: 2C',
       'Site: Face', 'Site: Lower.limb.and.hip',
       'Site: Scalp.and.neck', 'Site: Trunk', 'Site: Upper.limb.and.shoulder', 
       'Laterality: Left', 'Laterality: Midline', 'Laterality: Right',
       'Type: Lentigo.maligna',
       'Type: Melanoma.NOS', 'Type: Nodular',
       'Type: Superficial.spreading',
       'Ulceration: Absent', 'Ulceration: Present', 'Ulceration: Unavailable',
       'Precursor lesion: Present',
       'Precursor type: Absent',
       'Precursor type: Benign.nevus',
       'Precursor type: Dermal.nevus',
       'Precursor type: Dysplastic.nevus',
       'Precursor type: Lentigo.maligna',
        'Precursor type: Unknown.type',
        'Radial growth: Absent', 'Radial growth: Present','Radial growth: Unavailable',
        'Vertical growth: Absent',
        'Vertical growth: Present', 'Vertical growth: Unknown',
 'VGT: Absent',
 'VGT: Epithelioid',
 'VGT: Epithelioid.and.nevoid',
 'VGT: Epithelioid.and.small.cell',
 'VGT: Epithelioid.and.spindled',
 'VGT: Other', 'VGT: Small.cell',
 'VGT: Spindled', 'VGT: Unavailable',
 'VGT: Unknown.type',
 'Microsatellites: Absent',
 'Regression: Present',
 'TIL: Absent',
 'TIL: Present',
 'TIL: Unavailable',
 'TIL type: Absent',
 'TIL type: Brisk',
 'TIL type: Non.Brisk',
 'TIL type: Unknown',
 'TIL type: Unknown.type',      
       'Lymphovascular invasion: Present', 
        'Perineural invasion: Present',
       'margincheck_0', 'margincheck_1', 'margincheck_2', 
       'HNMSC: No', 'HNMSC: Unavailable', 'HNMSC: Yes', 
       'PSkin_situ_or_benigh_No',
       'PSkin_situ_or_benigh_Unavailable', 'PSkin_situ_or_benigh_Yes',
       'HOM: No', 'HOM: Unavailable', 'HOM: Yes',
       'HCAID: No', 'HCAID: Unavailable', 'HCAID: Yes', 
       'HSAID: No', 'HSAID: Unavailable', 'HSAID: Yes', 
       'RLNH: Not indicated', 'RLNH: Unknown reasean', 'RLNH: Age/Comorbidity',
       'RLNH: Deferred', 'RLNH: Negative'
]

In [ ]:
print(len(feature_names))

In [ ]:
n_repeats = 50
TOP = 20
feature_names = np.array(feature_names)

In [ ]:
if True:
    model = RandomSurvivalForest(bootstrap=False)

    param_grid = {'n_estimators': [100],
                  'min_samples_split': [4, 5, 7, 10, 12],
                  'min_samples_leaf': [4, 5, 6, 7, 8, 9, 10]
                 }
    cv = RepeatedStratifiedKFold(n_splits=nFold, n_repeats=1, random_state=42)
    gsearch = GridSearchCV(model, param_grid, scoring=score_survival_model, cv=cv)
    gsearch = gsearch.fit(X_mgh, y_mgh)

    print(gsearch.best_params_, gsearch.best_score_)

In [ ]:
model = RandomSurvivalForest(n_estimators=gsearch_RF.best_params_['n_estimators'],
                              min_samples_split=gsearch_RF.best_params_['min_samples_split'],
                              min_samples_leaf=gsearch_RF.best_params_['min_samples_leaf'])

model.fit(X_mgh, y_mgh)
result_RF = permutation_importance(model, X_mgh, y_mgh, 
                                   scoring = score_survival_model,
                                   n_repeats=n_repeats)
perm_sorted_idx_RF = result_RF.importances_mean.argsort()

In [ ]:
result = result_RF.importances[perm_sorted_idx_RF].T[:,-TOP:]
labels = (feature_names[perm_sorted_idx_RF])[-TOP:]

fig, ax = plt.subplots(figsize=(3,5),dpi=216)

plt.boxplot(result,
    vert=False,
    labels=labels,
)
plt.xlabel("Feature importance\n(RF-T)")
plt.show()

In [ ]:
if True:
    model = GradientBoostingSurvivalAnalysis(loss = 'coxph')

    param_grid = {'n_estimators': [100, 150, 200],
                  'learning_rate': [0.1, 0.2, 0.3, 0.7, 0.9, 1.0],
                  'max_depth': [1,2,3,4,5,6,7]
                 }
    cv = RepeatedStratifiedKFold(n_splits=nFold, n_repeats=1, random_state=42)
    gsearch = GridSearchCV(model, param_grid, scoring=score_survival_model,cv=cv)
    gsearch = gsearch.fit(X_mgh, y_mgh)

    print(gsearch.best_params_, gsearch.best_score_)

In [ ]:
model = GradientBoostingSurvivalAnalysis(
            loss = 'coxph', 
            n_estimators=gsearch.best_params_['n_estimators'], 
            learning_rate=gsearch.best_params_['learning_rate'], 
            max_depth=gsearch.best_params_['max_depth']
        )

model.fit(X_mgh, y_mgh)
result_GB = permutation_importance(model, X_mgh, y_mgh, 
                                   scoring = score_survival_model,
                                   n_repeats=n_repeats)
perm_sorted_idx_GB = result_GB.importances_mean.argsort()

In [ ]:
result = result_GB.importances[perm_sorted_idx_GB].T[:,-TOP:]
labels=(feature_names[perm_sorted_idx_GB])[-TOP:]

fig, ax = plt.subplots(figsize=(3,5),dpi=216)

plt.boxplot(
    result,
    vert=False,
    labels=labels,
)
plt.xlabel("Feature importance\n(GB-T)")
plt.show()

In [ ]:
if True:
    model = CoxnetSurvivalAnalysis()

    param_grid = {'l1_ratio': [0.01, 0.02, 0.03, 0.04, 0.05, 0.07, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6]
                 }
    cv = RepeatedStratifiedKFold(n_splits=nFold, n_repeats=1, random_state=42)
    gsearch = GridSearchCV(model, param_grid, scoring=score_survival_model,cv=cv)
    gsearch = gsearch.fit(X_mgh, y_mgh)

    print(gsearch.best_params_, gsearch.best_score_)
    gsearch_Coxnet = gsearch

In [ ]:
model = CoxnetSurvivalAnalysis(l1_ratio=gsearch_Coxnet.best_params_['l1_ratio'],
                                       fit_baseline_model=True)

model.fit(X_mgh, y_mgh)
result_Coxnet = permutation_importance(model, X_mgh, y_mgh, 
                                   scoring = score_survival_model,
                                   n_repeats=n_repeats)
perm_sorted_idx_Conxnet = result_Coxnet.importances_mean.argsort()

In [ ]:
result = result_Coxnet.importances[perm_sorted_idx_Conxnet].T[:,-TOP:]
labels=(feature_names[perm_sorted_idx_Conxnet])[-TOP:]

fig, ax = plt.subplots(figsize=(3,5),dpi=216)

plt.boxplot(
    result,
    vert=False,
    labels=labels,
)
plt.xlabel("Feature importance\n(Coxnet)")
plt.show() 

In [ ]:
if True:
    model = CoxPHSurvivalAnalysis()

    param_grid = {'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                  'n_iter': [70, 100, 150]
                 }
    
    cv = RepeatedStratifiedKFold(n_splits=nFold, n_repeats=1, random_state=42)
    gsearch = GridSearchCV(model, param_grid, scoring=score_survival_model,cv=cv)
    gsearch = gsearch.fit(X_mgh, y_mgh)

    print(gsearch.best_params_, gsearch.best_score_)
    gsearch_CoxPH = gsearch

In [ ]:
model = CoxPHSurvivalAnalysis(alpha=gsearch_CoxPH.best_params_['alpha'],
                                      n_iter=gsearch_CoxPH.best_params_['n_iter']
                                     )

model.fit(X_mgh, y_mgh)
result_CoxPH = permutation_importance(model, X_mgh, y_mgh, 
                                   scoring = score_survival_model,
                                   n_repeats=n_repeats)
perm_sorted_idx_CoxPH = result_CoxPH.importances_mean.argsort()

In [ ]:
result = result_CoxPH.importances[perm_sorted_idx_CoxPH].T[:,-TOP:]
labels=(feature_names[perm_sorted_idx_CoxPH])[-TOP:]

fig, ax = plt.subplots(figsize=(3,5),dpi=216)

plt.boxplot(
    result,
    vert=False,
    labels=labels,
)
plt.xlabel("Feature importance\n(CoxPH)")
plt.show()